In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark
from delta import *
import os

In [2]:

# Build Spark session with Delta configurations and Hive support
builder = SparkSession.builder.appName("bronzetosilver") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.io.native.lib.available", "true") \
    .config("spark.jars.packages", "org.apache.hive:hive-exec:2.3.9")  # Include Hive dependency

# Enable Hive support explicitly and get Spark session
spark = configure_spark_with_delta_pip(builder.enableHiveSupport()).getOrCreate()

# Optionally, you can test by running a Hive query
spark.sql("CREATE TABLE IF NOT EXISTS test_hive_table (name STRING, age INT) USING hive")
spark.sql("SELECT * FROM test_hive_table")


DataFrame[name: string, age: int]

In [3]:
# List of external bronze tables
bronze_tables = [
    "bronze.location",
    "bronze.payment",
    "bronze.payment_method",
    "bronze.payment_status",
    "bronze.request",
    "bronze.trip",
    "bronze.user",
    "bronze.vehicle",
    "bronze.vehicle_make"
] 
print(bronze_tables)

['bronze.location', 'bronze.payment', 'bronze.payment_method', 'bronze.payment_status', 'bronze.request', 'bronze.trip', 'bronze.user', 'bronze.vehicle', 'bronze.vehicle_make']


In [4]:
landing_location = os.path.join(os.getcwd())
bronze_location = os.path.join(landing_location, "bronze")
silver_location = fr"{os.getcwd()}/silver"
silver_tables = [bronze_tables.replace("bronze", "silver") for bronze_tables in bronze_tables]
print(silver_tables)
print(os.getcwd())
print(bronze_location)

['silver.location', 'silver.payment', 'silver.payment_method', 'silver.payment_status', 'silver.request', 'silver.trip', 'silver.user', 'silver.vehicle', 'silver.vehicle_make']
e:\DEPIfINALpROJECT\DATABASEcSV
e:\DEPIfINALpROJECT\DATABASEcSV\bronze


In [5]:
# Verify the current schema
print("Current Schema:")
spark.sql("SELECT current_database()")

# List all tables in the bronze schema
print("Tables in bronze schema:")
spark.sql("SHOW TABLES IN bronze").show()
spark.sql("SELECT * FROM bronze.location LIMIT 1").show()
spark.sql("SELECT * FROM bronze.payment LIMIT 1").show()


# Describe the history of the Delta table
print("Describing history for bronze.location:")




Current Schema:
Tables in bronze schema:
+---------+---------------+-----------+
|namespace|      tableName|isTemporary|
+---------+---------------+-----------+
|   bronze|       location|      false|
|   bronze|        payment|      false|
|   bronze|  paymentmethod|      false|
|   bronze|  paymentstatus|      false|
|   bronze|        request|      false|
|   bronze|test_hive_table|      false|
|   bronze|           trip|      false|
|   bronze|           user|      false|
|   bronze|   vehiclemakes|      false|
|   bronze|       vehicles|      false|
+---------+---------------+-----------+

+----------+------------+-----------+-------------------+--------------------+-----------------------------+
|LocationID|   Longitude|   Latitude|   _processing_date|     _input_filename|_input_file_modification_date|
+----------+------------+-----------+-------------------+--------------------+-----------------------------+
|    568573|-73.77675629|40.64520645|2024-10-10 17:44:46|file:///e:/DEP

In [6]:

deltaTable = DeltaTable.forPath(spark, "bronze\\location_df_raw")
fullHistoryDF = deltaTable.history()
fullHistoryDF.show(truncate=False)

+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------+----+--------+---------+-----------+-----------------+-------------+----------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                 |job |notebook|clusterId|readVersion|isolationLevel   |isBlindAppend|operationMetrics                                                      |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------+----+--------+---------+-----------+-----------------+-------------+----------------------------------------------------------------------+------------+-----------------------------------+
|18     |2024-10-11 09:10:26.004|NULL  |

In [7]:
from pyspark.sql.functions import to_date, current_date

# Read data from each table and assign to individual variables
user_df = spark.read.table("bronze.user").filter(to_date("_processing_date") == current_date())
location_df = spark.read.table("bronze.location").filter(to_date("_processing_date") == current_date())
payment_df = spark.read.table("bronze.payment").filter(to_date("_processing_date") == current_date())
paymentmethod_df = spark.read.table("bronze.paymentmethod").filter(to_date("_processing_date") == current_date())
paymentstatus_df = spark.read.table("bronze.paymentstatus").filter(to_date("_processing_date") == current_date())
request_df = spark.read.table("bronze.request").filter(to_date("_processing_date") == current_date())
trip_df = spark.read.table("bronze.trip").filter(to_date("_processing_date") == current_date())
vehicle_df = spark.read.table("bronze.vehicles").filter(to_date("_processing_date") == current_date())
vehiclemakes_df = spark.read.table("bronze.vehiclemakes").filter(to_date("_processing_date") == current_date())

# Show each DataFrame separately
user_df.show()
location_df.show()
payment_df.show()
paymentmethod_df.show()
paymentstatus_df.show()
request_df.show()
trip_df.show()
vehicle_df.show()
vehiclemakes_df.show()




+------+--------------------+--------------------+--------------+----------------+-------------------+--------------------+-----------------------------+
|UserID|            FullName|               Email|   PhoneNumber|DriverMeanRating|   _processing_date|     _input_filename|_input_file_modification_date|
+------+--------------------+--------------------+--------------+----------------+-------------------+--------------------+-----------------------------+
|975676|      Frank Gonzalez| markkim@example.net|(997) 243-7176|            NULL|2024-10-11 08:21:59|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|975676|        Nicole Jones|theresa03@example...|(115) 458-5338|            NULL|2024-10-11 08:21:59|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|975678|        Becky Watson|tinacunningham@ex...|(771) 305-4349|            NULL|2024-10-11 08:21:59|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|975679| Stephanie Robertson|teresahall@exampl...|(174) 369-7237|           

In [8]:
# Function to load delta tables into dataframes
#def delta_to_df(file_names):

    # Create an empty dictionary to hold the DataFrames
    #dataframes = {}

    # Loop over the filenames and load each CSV into a DataFrame
    #for file_name in file_names:
        #dataframes[file_name] = DeltaTable.forPath(spark, f"bronze\\{file_name}").toDF()
     
    #return dataframes

In [9]:
#dataframes = delta_to_df(file_names)
#for df_name, df in dataframes.items():
    #globals()[df_name] = df

# Data Cleaning (Drop NULL Values)

In [10]:
user_df = user_df.drop("DriverMeanRating")
trip_df = trip_df.drop("driver_rating")

In [11]:
user_df.show()


+------+--------------------+--------------------+--------------+-------------------+--------------------+-----------------------------+
|UserID|            FullName|               Email|   PhoneNumber|   _processing_date|     _input_filename|_input_file_modification_date|
+------+--------------------+--------------------+--------------+-------------------+--------------------+-----------------------------+
|975676|      Frank Gonzalez| markkim@example.net|(997) 243-7176|2024-10-11 08:21:59|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|975676|        Nicole Jones|theresa03@example...|(115) 458-5338|2024-10-11 08:21:59|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|975678|        Becky Watson|tinacunningham@ex...|(771) 305-4349|2024-10-11 08:21:59|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|975679| Stephanie Robertson|teresahall@exampl...|(174) 369-7237|2024-10-11 08:21:59|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|975680|      Clarence Olson|graydebra@ex

In [12]:
#user_df.groupBy("FullName", "Email", "PhoneNumber").count().orderBy(desc("count")).show()

In [13]:
location_df.show()

+----------+------------+-----------+-------------------+--------------------+-----------------------------+
|LocationID|   Longitude|   Latitude|   _processing_date|     _input_filename|_input_file_modification_date|
+----------+------------+-----------+-------------------+--------------------+-----------------------------+
|   1509808|-73.98127747|40.74430084|2024-10-11 08:21:46|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|   1498812|-73.96249390|40.77143097|2024-10-11 08:21:46|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|   1021474|-73.99668121|40.71631622|2024-10-11 08:21:46|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|   1655657|-73.96881866|40.76142883|2024-10-11 08:21:46|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|   1324161|-73.98605347|40.75881577|2024-10-11 08:21:46|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|   1109136|-73.98454285|40.75430298|2024-10-11 08:21:46|file:///e:/DEPIfI...|         2024-10-08 09:46:...|
|    836603|-73.975

In [14]:
#location_df.distinct().count()

In [15]:
#location_df.groupBy("Longitude", "Latitude").count().orderBy(desc("count")).show()

In [16]:
"""

from pyspark.sql import Window
from pyspark.sql.functions import count, row_number, desc

# Define the window specification for counting
window_spec = Window.partitionBy("Longitude", "Latitude").orderBy(desc("count"))

# Group by Longitude and Latitude, count occurrences, and order
location_with_count = location_df.groupBy("Longitude", "Latitude") \
    .agg(count("*").alias("count"))

# Define a new window to assign row numbers based on count
rownum_window = Window.orderBy(desc("count"))

# Add a row number based on the ordered counts
location_with_rownum = location_with_count.withColumn("rownum", row_number().over(rownum_window))

# Filter to keep only rows with count equal to 1
filtered_df = location_with_rownum.filter(location_with_rownum["count"] == 1)

# Show the results, ordering by count
filtered_df.select("Longitude", "Latitude", "count").orderBy(desc("count")).show()
"""

'\n\nfrom pyspark.sql import Window\nfrom pyspark.sql.functions import count, row_number, desc\n\n# Define the window specification for counting\nwindow_spec = Window.partitionBy("Longitude", "Latitude").orderBy(desc("count"))\n\n# Group by Longitude and Latitude, count occurrences, and order\nlocation_with_count = location_df.groupBy("Longitude", "Latitude")     .agg(count("*").alias("count"))\n\n# Define a new window to assign row numbers based on count\nrownum_window = Window.orderBy(desc("count"))\n\n# Add a row number based on the ordered counts\nlocation_with_rownum = location_with_count.withColumn("rownum", row_number().over(rownum_window))\n\n# Filter to keep only rows with count equal to 1\nfiltered_df = location_with_rownum.filter(location_with_rownum["count"] == 1)\n\n# Show the results, ordering by count\nfiltered_df.select("Longitude", "Latitude", "count").orderBy(desc("count")).show()\n'

In [17]:
#location_df.count()


In [18]:
#filtered_df.count()

In [19]:
#filtered_df.groupBy("Longitude", "Latitude").count().orderBy(desc("count")).show()

In [20]:
trip_df.show(10)

+------+---------+--------+---------+---------+--------------------+--------------------+------------+--------+---------+------+---------+-----------+--------------------+-------------------+--------------------+-----------------------------+
|TripID|RequestID|DriverID|VehicleID|PaymentID|       TripStartTime|         TripEndTime|TripDistance|BaseFare|ExtraFare|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|   _processing_date|     _input_filename|_input_file_modification_date|
+------+---------+--------+---------+---------+--------------------+--------------------+------------+--------+---------+------+---------+-----------+--------------------+-------------------+--------------------+-----------------------------+
| 52237|   177781|  809410|   309496|        4|2015-01-10 15:27:...|2015-01-10 15:33:...|        1.80|    7.50|      .00|   .50|      .00|        .00|                 .30|2024-10-11 08:21:56|file:///e:/DEPIfI...|         2024-10-08 09:47:...|
| 52238|   130888|  881048| 

In [21]:

user_df.printSchema()
location_df.printSchema()
payment_df.printSchema()
paymentmethod_df.printSchema()
paymentstatus_df.printSchema()
request_df.printSchema()
trip_df.printSchema()
vehicle_df.printSchema()
vehiclemakes_df.printSchema()

root
 |-- UserID: string (nullable = true)
 |-- FullName: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- PhoneNumber: string (nullable = true)
 |-- _processing_date: timestamp (nullable = true)
 |-- _input_filename: string (nullable = true)
 |-- _input_file_modification_date: timestamp (nullable = true)

root
 |-- LocationID: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- _processing_date: timestamp (nullable = true)
 |-- _input_filename: string (nullable = true)
 |-- _input_file_modification_date: timestamp (nullable = true)

root
 |-- PaymentID: string (nullable = true)
 |-- PaymentMethodID: string (nullable = true)
 |-- PaymentStatusID: string (nullable = true)
 |-- _processing_date: timestamp (nullable = true)
 |-- _input_filename: string (nullable = true)
 |-- _input_file_modification_date: timestamp (nullable = true)

root
 |-- PaymentMethodID: string (nullable = true)
 |-- MethodName: string (nul

In [22]:
# Define schemas
user_schema = StructType([
    StructField("UserID", IntegerType(), True),
    StructField("FullName", StringType(), True),
    StructField("Email", StringType(), True),
    StructField("PhoneNumber", StringType(), True),
    StructField("_processing_date", TimestampType(), True),
    StructField("_input_filename", StringType(), True),
    StructField("_input_file_modification_date", TimestampType(), True)
])

location_schema = StructType([
    StructField("LocationID", IntegerType(), True),
    StructField("Longitude", DoubleType(), True),
    StructField("Latitude", DoubleType(), True),
    StructField("_processing_date", TimestampType(), True),
    StructField("_input_filename", StringType(), True),
    StructField("_input_file_modification_date", TimestampType(), True)
])

payment_schema = StructType([
    StructField("PaymentID", IntegerType(), True),
    StructField("PaymentMethodID", IntegerType(), True),
    StructField("PaymentStatusID", IntegerType(), True),
    StructField("_processing_date", TimestampType(), True),
    StructField("_input_filename", StringType(), True),
    StructField("_input_file_modification_date", TimestampType(), True)
])

paymentmethod_schema = StructType([
    StructField("PaymentMethodID", IntegerType(), True),
    StructField("MethodName", StringType(), True),
    StructField("_processing_date", TimestampType(), True),
    StructField("_input_filename", StringType(), True),
    StructField("_input_file_modification_date", TimestampType(), True)
])

paymentstatus_schema = StructType([
    StructField("PaymentStatusID", IntegerType(), True),
    StructField("StatusName", StringType(), True),
    StructField("_processing_date", TimestampType(), True),
    StructField("_input_filename", StringType(), True),
    StructField("_input_file_modification_date", TimestampType(), True)
])

request_schema = StructType([
    StructField("RequestID", IntegerType(), True),
    StructField("PassengerID", IntegerType(), True),
    StructField("PickupLocationID", IntegerType(), True),
    StructField("DropoffLocationID", IntegerType(), True),
    StructField("RequestTime", TimestampType(), True),  # Convert to TimestampType
    StructField("AcceptTime", TimestampType(), True),   # Convert to TimestampType
    StructField("_processing_date", TimestampType(), True),
    StructField("_input_filename", StringType(), True),
    StructField("_input_file_modification_date", TimestampType(), True)
])

trip_schema = StructType([
    StructField("TripID", IntegerType(), True),
    StructField("RequestID", IntegerType(), True),
    StructField("DriverID", IntegerType(), True),
    StructField("VehicleID", IntegerType(), True),
    StructField("PaymentID", IntegerType(), True),
    StructField("TripStartTime", TimestampType(), True),  # Convert to TimestampType
    StructField("TripEndTime", TimestampType(), True),    # Convert to TimestampType
    StructField("TripDistance", DoubleType(), True),
    StructField("BaseFare", DoubleType(), True),
    StructField("ExtraFare", DoubleType(), True),
    StructField("MtaTax", DoubleType(), True),
    StructField("TipAmount", DoubleType(), True),
    StructField("TollsAmount", DoubleType(), True),
    StructField("ImprovementSurcharge", DoubleType(), True),
    StructField("_processing_date", TimestampType(), True),
    StructField("_input_filename", StringType(), True),
    StructField("_input_file_modification_date", TimestampType(), True)
])

vehicle_schema = StructType([
    StructField("VehicleID", IntegerType(), True),
    StructField("DriverID", IntegerType(), True),
    StructField("MakeID", IntegerType(), True),
    StructField("Model", StringType(), True),
    StructField("Year", IntegerType(), True),
    StructField("Color", StringType(), True),
    StructField("LicensePlate", StringType(), True),
    StructField("_processing_date", TimestampType(), True),
    StructField("_input_filename", StringType(), True),
    StructField("_input_file_modification_date", TimestampType(), True)
])

vehiclemakes_schema = StructType([
    StructField("MakeID", IntegerType(), True),
    StructField("MakeName", StringType(), True),
    StructField("_processing_date", TimestampType(), True),
    StructField("_input_filename", StringType(), True),
    StructField("_input_file_modification_date", TimestampType(), True)
])

In [23]:
# Define a function to convert the DataFrame
def convert_data_types(df, conversion_map):
    for column_name, target_type in conversion_map.items():
        df = df.withColumn(column_name, col(column_name).cast(target_type))
    return df

In [24]:
# Define conversion maps
users_conversion = {
    "UserID": IntegerType(),
    "FullName": StringType(),
    "Email": StringType(),
    "PhoneNumber": StringType(),
    "_processing_date": TimestampType(),
    "_input_filename": StringType(),
    "_input_file_modification_date": TimestampType()
}

locations_conversion = {
    "LocationID": IntegerType(),
    "Longitude": DoubleType(),
    "Latitude": DoubleType(),
    "_processing_date": TimestampType(),
    "_input_filename": StringType(),
    "_input_file_modification_date": TimestampType()
}

payments_conversion = {
    "PaymentID": IntegerType(),
    "PaymentMethodID": IntegerType(),
    "PaymentStatusID": IntegerType(),
    "_processing_date": TimestampType(),
    "_input_filename": StringType(),
    "_input_file_modification_date": TimestampType()
}

payment_methods_conversion = {
    "PaymentMethodID": IntegerType(),
    "MethodName": StringType(),
    "_processing_date": TimestampType(),
    "_input_filename": StringType(),
    "_input_file_modification_date": TimestampType()
}

payment_statuses_conversion = {
    "PaymentStatusID": IntegerType(),
    "StatusName": StringType(),
    "_processing_date": TimestampType(),
    "_input_filename": StringType(),
    "_input_file_modification_date": TimestampType()
}

requests_conversion = {
    "RequestID": IntegerType(),
    "PassengerID": IntegerType(),
    "PickupLocationID": IntegerType(),
    "DropoffLocationID": IntegerType(),
    "RequestTime": TimestampType(),  # Will be converted later
    "AcceptTime": TimestampType(),   # Will be converted later
    "_processing_date": TimestampType(),
    "_input_filename": StringType(),
    "_input_file_modification_date": TimestampType()
}

trips_conversion = {
    "TripID": IntegerType(),
    "RequestID": IntegerType(),
    "DriverID": IntegerType(),
    "VehicleID": IntegerType(),
    "PaymentID": IntegerType(),
    "TripStartTime": TimestampType(),  # Will be converted later
    "TripEndTime": TimestampType(),    # Will be converted later
    "TripDistance": DoubleType(),
    "BaseFare": DoubleType(),
    "ExtraFare": DoubleType(),
    "MtaTax": DoubleType(),
    "TipAmount": DoubleType(),
    "TollsAmount": DoubleType(),
    "ImprovementSurcharge": DoubleType(),
    "_processing_date": TimestampType(),
    "_input_filename": StringType(),
    "_input_file_modification_date": TimestampType()
}

vehicles_conversion = {
    "VehicleID": IntegerType(),
    "DriverID": IntegerType(),
    "MakeID": IntegerType(),
    "Model": StringType(),
    "Year": IntegerType(),
    "Color": StringType(),
    "LicensePlate": StringType(),
    "_processing_date": TimestampType(),
    "_input_filename": StringType(),
    "_input_file_modification_date": TimestampType()
}

makes_conversion = {
    "MakeID": IntegerType(),
    "MakeName": StringType(),
    "_processing_date": TimestampType(),
    "_input_filename": StringType(),
    "_input_file_modification_date": TimestampType()
}


In [25]:
# Convert the data types for each DataFrame
user_df = convert_data_types(user_df, users_conversion)
location_df = convert_data_types(location_df, locations_conversion)
payment_df = convert_data_types(payment_df, payments_conversion)
paymentmethod_df = convert_data_types(paymentmethod_df, payment_methods_conversion)
paymentstatus_df = convert_data_types(paymentstatus_df, payment_statuses_conversion)
request_df = convert_data_types(request_df, requests_conversion)
trip_df = convert_data_types(trip_df, trips_conversion)
vehicle_df = convert_data_types(vehicle_df, vehicles_conversion)
vehiclemakes_df = convert_data_types(vehiclemakes_df, makes_conversion)

In [26]:
user_df.printSchema()
location_df.printSchema()
payment_df.printSchema()
paymentmethod_df.printSchema()
paymentstatus_df.printSchema()
request_df.printSchema()
trip_df.printSchema()
vehicle_df.printSchema()
vehiclemakes_df.printSchema()

root
 |-- UserID: integer (nullable = true)
 |-- FullName: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- PhoneNumber: string (nullable = true)
 |-- _processing_date: timestamp (nullable = true)
 |-- _input_filename: string (nullable = true)
 |-- _input_file_modification_date: timestamp (nullable = true)

root
 |-- LocationID: integer (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- _processing_date: timestamp (nullable = true)
 |-- _input_filename: string (nullable = true)
 |-- _input_file_modification_date: timestamp (nullable = true)

root
 |-- PaymentID: integer (nullable = true)
 |-- PaymentMethodID: integer (nullable = true)
 |-- PaymentStatusID: integer (nullable = true)
 |-- _processing_date: timestamp (nullable = true)
 |-- _input_filename: string (nullable = true)
 |-- _input_file_modification_date: timestamp (nullable = true)

root
 |-- PaymentMethodID: integer (nullable = true)
 |-- MethodName: strin

In [27]:
trip_df.show()

+------+---------+--------+---------+---------+-------------------+-------------------+------------+--------+---------+------+---------+-----------+--------------------+-------------------+--------------------+-----------------------------+
|TripID|RequestID|DriverID|VehicleID|PaymentID|      TripStartTime|        TripEndTime|TripDistance|BaseFare|ExtraFare|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|   _processing_date|     _input_filename|_input_file_modification_date|
+------+---------+--------+---------+---------+-------------------+-------------------+------------+--------+---------+------+---------+-----------+--------------------+-------------------+--------------------+-----------------------------+
| 52237|   177781|  809410|   309496|        4|2015-01-10 15:27:00|2015-01-10 15:33:00|         1.8|     7.5|      0.0|   0.5|      0.0|        0.0|                 0.3|2024-10-11 08:21:56|file:///e:/DEPIfI...|         2024-10-08 09:47:...|
| 52238|   130888|  881048|    60732

# Handling Missing Vlaues

In [28]:
#trip_df = trip_df.na.fill({"BaseFare": 0, "ExtraFare": 0, "MtaTax": 0, "TipAmount": 0, "TollsAmount": 0, "ImprovementSurcharge": 0})

In [29]:
trip_df.show(5)

+------+---------+--------+---------+---------+-------------------+-------------------+------------+--------+---------+------+---------+-----------+--------------------+-------------------+--------------------+-----------------------------+
|TripID|RequestID|DriverID|VehicleID|PaymentID|      TripStartTime|        TripEndTime|TripDistance|BaseFare|ExtraFare|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|   _processing_date|     _input_filename|_input_file_modification_date|
+------+---------+--------+---------+---------+-------------------+-------------------+------------+--------+---------+------+---------+-----------+--------------------+-------------------+--------------------+-----------------------------+
| 52237|   177781|  809410|   309496|        4|2015-01-10 15:27:00|2015-01-10 15:33:00|         1.8|     7.5|      0.0|   0.5|      0.0|        0.0|                 0.3|2024-10-11 08:21:56|file:///e:/DEPIfI...|         2024-10-08 09:47:...|
| 52238|   130888|  881048|    60732

In [30]:
trip_df.printSchema()

root
 |-- TripID: integer (nullable = true)
 |-- RequestID: integer (nullable = true)
 |-- DriverID: integer (nullable = true)
 |-- VehicleID: integer (nullable = true)
 |-- PaymentID: integer (nullable = true)
 |-- TripStartTime: timestamp (nullable = true)
 |-- TripEndTime: timestamp (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- BaseFare: double (nullable = true)
 |-- ExtraFare: double (nullable = true)
 |-- MtaTax: double (nullable = true)
 |-- TipAmount: double (nullable = true)
 |-- TollsAmount: double (nullable = true)
 |-- ImprovementSurcharge: double (nullable = true)
 |-- _processing_date: timestamp (nullable = true)
 |-- _input_filename: string (nullable = true)
 |-- _input_file_modification_date: timestamp (nullable = true)



# Converting Date columns

#### modify this cell for needed dataframes, columns  

In [31]:
'''
# Define the mapping of DataFrames to their respective columns for timestamp conversion
dataframes_with_columns = {
    "trip_df": (trip_df, ["TripStartTime", "TripEndTime"]),
    "request_df": (request_df, ["RequestTime", "AcceptTime"]),
    # Add more DataFrames and their columns as needed
}
'''


'\n# Define the mapping of DataFrames to their respective columns for timestamp conversion\ndataframes_with_columns = {\n    "trip_df": (trip_df, ["TripStartTime", "TripEndTime"]),\n    "request_df": (request_df, ["RequestTime", "AcceptTime"]),\n    # Add more DataFrames and their columns as needed\n}\n'

##### converting to date and verifying the result

In [32]:
'''
from pyspark.sql.functions import to_timestamp, col

# Define the mapping of DataFrames to their respective columns for timestamp conversion
dataframes_with_columns = {
    "trip_df": (trip_df, ["TripStartTime", "TripEndTime"]),
    "request_df": (request_df, ["RequestTime", "AcceptTime"]),
    # Add more DataFrames and their columns as needed
}

# Function to convert timestamp columns
def convert_timestamp_columns(df, columns, date_format="M/d/yyyy h:mm:ss a"):
    for col_name in columns:
        df = df.withColumn(col_name, to_timestamp(col(col_name), date_format).cast("date"))
    return df

# Loop through each DataFrame, apply the conversion, and replace the old DataFrame directly
for table_name, (df, columns) in dataframes_with_columns.items():
    # Convert timestamp columns
    updated_df = convert_timestamp_columns(df, columns)
    
    # Replace the old DataFrame directly by updating the global variables
    globals()[table_name] = updated_df
    
    # Automatically show the updated DataFrame and print the schema
    print(f"DataFrame: {table_name}")
    updated_df.show(5)
    updated_df.printSchema()
'''


'\nfrom pyspark.sql.functions import to_timestamp, col\n\n# Define the mapping of DataFrames to their respective columns for timestamp conversion\ndataframes_with_columns = {\n    "trip_df": (trip_df, ["TripStartTime", "TripEndTime"]),\n    "request_df": (request_df, ["RequestTime", "AcceptTime"]),\n    # Add more DataFrames and their columns as needed\n}\n\n# Function to convert timestamp columns\ndef convert_timestamp_columns(df, columns, date_format="M/d/yyyy h:mm:ss a"):\n    for col_name in columns:\n        df = df.withColumn(col_name, to_timestamp(col(col_name), date_format).cast("date"))\n    return df\n\n# Loop through each DataFrame, apply the conversion, and replace the old DataFrame directly\nfor table_name, (df, columns) in dataframes_with_columns.items():\n    # Convert timestamp columns\n    updated_df = convert_timestamp_columns(df, columns)\n    \n    # Replace the old DataFrame directly by updating the global variables\n    globals()[table_name] = updated_df\n    \n  

# Renaming all columns in each dataframe to snake_case

In [33]:
import re
from pyspark.sql import DataFrame

# Function to convert a string to snake_case
def to_snake_case(name: str) -> str:
    # Replace spaces and hyphens with underscores
    name = re.sub(r'[\s-]+', '_', name)

    # Insert underscores before any uppercase letter that follows a lowercase letter
    name = re.sub(r'([a-z])([A-Z])', r'\1_\2', name)

    # Insert underscores between groups of uppercase letters and lowercase letters
    name = re.sub(r'([A-Z]+)([A-Z][a-z])', r'\1_\2', name)  # e.g., ABc -> AB_c
    
    # Handle multiple uppercase letters
    name = re.sub(r'([A-Z]+)([A-Z][a-z])', r'\1_\2', name)  # Ensure a transition from uppercase to lowercase has an underscore

    # Avoid double underscores by replacing multiple underscores with a single underscore
    name = re.sub(r'__+', '_', name)

    # Remove leading underscores if present and convert to lowercase
    name = name.lstrip('_').lower()

    return name

# Function to rename all columns in a DataFrame to snake_case
def convert_columns_to_snake_case(df: DataFrame) -> DataFrame:
    # Get the existing column names
    original_columns = df.columns
    # Generate new column names in snake_case
    new_columns = [to_snake_case(col) for col in original_columns]

    # Rename the columns in the DataFrame
    for original_col, new_col in zip(original_columns, new_columns):
        df = df.withColumnRenamed(original_col, new_col)
    return df



In [34]:
# List of DataFrames
dataframes = [user_df, location_df, payment_df, paymentmethod_df, paymentstatus_df, request_df, trip_df, vehicle_df, vehiclemakes_df]
# Names of the original DataFrames
dataframe_names = ['user_df', 'location_df', 'payment_df', 'paymentmethod_df', 'paymentstatus_df', 'request_df', 'trip_df', 'vehicle_df', 'vehiclemakes_df']


In [35]:
# Loop through the list of DataFrames and apply the column name conversion
for i, df in enumerate(dataframes):
    # Apply column name conversion
    dataframes[i] = convert_columns_to_snake_case(df)
    
    # Reassign the modified DataFrame back to the original global variable
    globals()[dataframe_names[i]] = dataframes[i]
    
    # Show the first 5 rows of the updated DataFrame
    dataframes[i].show(5)


+-------+-------------------+--------------------+--------------+-------------------+--------------------+----------------------------+
|user_id|          full_name|               email|  phone_number|    processing_date|      input_filename|input_file_modification_date|
+-------+-------------------+--------------------+--------------+-------------------+--------------------+----------------------------+
| 975676|     Frank Gonzalez| markkim@example.net|(997) 243-7176|2024-10-11 08:21:59|file:///e:/DEPIfI...|        2024-10-08 09:46:...|
| 975676|       Nicole Jones|theresa03@example...|(115) 458-5338|2024-10-11 08:21:59|file:///e:/DEPIfI...|        2024-10-08 09:46:...|
| 975678|       Becky Watson|tinacunningham@ex...|(771) 305-4349|2024-10-11 08:21:59|file:///e:/DEPIfI...|        2024-10-08 09:46:...|
| 975679|Stephanie Robertson|teresahall@exampl...|(174) 369-7237|2024-10-11 08:21:59|file:///e:/DEPIfI...|        2024-10-08 09:46:...|
| 975680|     Clarence Olson|graydebra@example..

In [36]:
trip_df.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+---------------------+--------------------+
|summary|          trip_id|        request_id|         driver_id|        vehicle_id|        payment_id|     trip_distance|         base_fare|        extra_fare|            mta_tax|        tip_amount|       tolls_amount|improvement_surcharge|      input_filename|
+-------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+---------------------+--------------------+
|  count|           584875|            584875|            584875|            584875|            584875|            584875|            584875|            584875|             584875|            584875|            

In [95]:
processing_date = date_trunc('second', current_timestamp())

for i, df in enumerate(dataframes):
    # Apply column name conversion
    df = df.withColumn("processing_date", processing_date) \
    .withColumnRenamed("input_file_modification_date", "record_modified_date") \
    .drop("input_filename")

    # Reassign the modified DataFrame back to the original global variable
    globals()[dataframe_names[i]] = df
    
    # Show the first 5 rows of the updated DataFrame
trip_df.show()


+-------+----------+---------+----------+----------+-------------------+-------------------+-------------+---------+----------+-------+----------+------------+---------------------+-------------------+--------------------+
|trip_id|request_id|driver_id|vehicle_id|payment_id|    trip_start_time|      trip_end_time|trip_distance|base_fare|extra_fare|mta_tax|tip_amount|tolls_amount|improvement_surcharge|    processing_date|record_modified_date|
+-------+----------+---------+----------+----------+-------------------+-------------------+-------------+---------+----------+-------+----------+------------+---------------------+-------------------+--------------------+
|  52237|    177781|   809410|    309496|         4|2015-01-10 15:27:00|2015-01-10 15:33:00|          1.8|      7.5|       0.0|    0.5|       0.0|         0.0|                  0.3|2024-10-11 17:25:04|2024-10-08 09:47:...|
|  52238|    130888|   881048|     60732|         4|2015-01-10 15:27:00|2015-01-10 15:34:00|          1.1|  

In [41]:
# List of DataFrames and their locations, including hash columns and SCD status
tables_info = [
    {"df": user_df, "location": f"{silver_location}/user", "hash_columns": ["user_id", "full_name", "email", "phone_number"], "is_scd": True},
    {"df": location_df, "location": f"{silver_location}/location", "hash_columns": ["location_id", "longitude", "latitude"], "is_scd": True},
    {"df": payment_df, "location": f"{silver_location}/payment", "hash_columns": ["payment_id", "payment_method_id", "payment_status_id"], "is_scd": False},  
    {"df": paymentmethod_df, "location": f"{silver_location}/payment_method", "hash_columns": ["payment_method_id", "method_name"], "is_scd": True},
    {"df": paymentstatus_df, "location": f"{silver_location}/payment_status", "hash_columns": ["payment_status_id", "status_name"], "is_scd": True},
    {"df": request_df, "location": f"{silver_location}/request", "hash_columns": ["request_id", "passenger_id", "pickup_location_id", "dropoff_location_id", "request_time", "accept_time"], "is_scd": False},
    {"df": trip_df, "location": f"{silver_location}/trip", "hash_columns": ["trip_id", "request_id", "driver_id", "vehicle_id", "payment_id", "trip_start_time", "trip_end_time", "trip_distance", "base_fare", "extra_fare", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge"], "is_scd": False},
    {"df": vehicle_df, "location": f"{silver_location}/vehicle", "hash_columns": ["vehicle_id", "driver_id", "make_id", "model", "year", "color", "license_plate"], "is_scd": True},
    {"df": vehiclemakes_df, "location": f"{silver_location}/vehicle_make", "hash_columns": ["make_id", "make_name"], "is_scd": True}
    # Add other DataFrames and locations here
]

# Loop through each table and perform merge with hash
for table_info in tables_info:
    source_df = table_info["df"]
    delta_location = table_info["location"]
    hash_columns = table_info["hash_columns"]  # Columns to hash
    is_scd = table_info["is_scd"]  # Check if it's a slowly changing dimension

    print(f"Processing DataFrame at {delta_location}...")

    # Generate hash column for all DataFrames
    print(f"Generating hash column for DataFrame at {delta_location}...")
    source_df_hashed = generate_hash_column(source_df, hash_columns)
    print(f"Hash column generated for {delta_location}.")

    try:
        # Check if the Delta table exists at the specified location
        if DeltaTable.isDeltaTable(spark, delta_location):
            print(f"Delta table found at {delta_location}. Proceeding with merge...")
            delta_table = DeltaTable.forPath(spark, delta_location)
            
            # Alias for target and source
            target_alias = "target"
            source_alias = "src"

            # If it's an SCD, proceed with merging
            if is_scd:
                delta_table_df = spark.read.format("delta").load(delta_location)
                delta_table_hashed = generate_hash_column(delta_table_df, hash_columns)
                # Build the merge condition using hash column
                merge_condition = build_merge_condition_with_hash(target_alias, source_alias, "record_hash")
                
                print(f"Merging data for {delta_location}...")
                # Perform the merge operation
                delta_table.alias(target_alias).merge(
                    source=source_df_hashed.alias(source_alias),
                    condition=merge_condition
                ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
            else:
                # If not an SCD, write the new DataFrame directly
                print(f"No merging needed for {delta_location}. Inserting data directly...")
                source_df_hashed.write.format("delta").mode("overwrite").save(delta_location)
            
            print(f"Processing completed for {delta_location}.")

        else:
            # If the Delta table doesn't exist, write the DataFrame to that location
            print(f"No Delta table found at {delta_location}. Writing new DataFrame...")
            source_df_hashed.write.mode("overwrite").format("delta").save(delta_location)
            print(f"DataFrame written to {delta_location}.")

    except Exception as e:
        print(f"Error processing table at {delta_location}: {e}")


Processing DataFrame at e:\DEPIfINALpROJECT\DATABASEcSV/silver/user...
Generating hash column for DataFrame at e:\DEPIfINALpROJECT\DATABASEcSV/silver/user...
Hash column generated for e:\DEPIfINALpROJECT\DATABASEcSV/silver/user.
Delta table found at e:\DEPIfINALpROJECT\DATABASEcSV/silver/user. Proceeding with merge...
Merging data for e:\DEPIfINALpROJECT\DATABASEcSV/silver/user...
Processing completed for e:\DEPIfINALpROJECT\DATABASEcSV/silver/user.
Processing DataFrame at e:\DEPIfINALpROJECT\DATABASEcSV/silver/location...
Generating hash column for DataFrame at e:\DEPIfINALpROJECT\DATABASEcSV/silver/location...
Hash column generated for e:\DEPIfINALpROJECT\DATABASEcSV/silver/location.
Delta table found at e:\DEPIfINALpROJECT\DATABASEcSV/silver/location. Proceeding with merge...
Merging data for e:\DEPIfINALpROJECT\DATABASEcSV/silver/location...
Processing completed for e:\DEPIfINALpROJECT\DATABASEcSV/silver/location.
Processing DataFrame at e:\DEPIfINALpROJECT\DATABASEcSV/silver/payme

In [61]:
spark.read.format("delta").load(f"{silver_location}/location").show()


+-----------+------------+-----------+-------------------+--------------------+--------------------+
|location_id|   longitude|   latitude|    processing_date|record_modified_date|         record_hash|
+-----------+------------+-----------+-------------------+--------------------+--------------------+
|      85038|-73.99800873|40.74245834|2024-10-11 16:52:48|2024-10-08 09:46:...|0000254e382162fca...|
|    1092867|-74.00403595| 40.7159462|2024-10-11 16:52:48|2024-10-08 09:46:...|0000bce1974a6b469...|
|    1474948|-73.96999359|40.78442764|2024-10-11 16:52:48|2024-10-08 09:46:...|0002d4a0b401e2422...|
|     997611|-73.98477936|40.74002838|2024-10-11 16:52:48|2024-10-08 09:46:...|0003e35f153b42508...|
|    1437020|-73.97665405|40.78242111|2024-10-11 16:52:48|2024-10-08 09:46:...|0003ec28a103d0f0f...|
|     567008|-73.78175354|40.64459229|2024-10-11 16:52:48|2024-10-08 09:46:...|0004271848b2bd1ae...|
|     454208|-73.96318054|40.79439545|2024-10-11 16:52:48|2024-10-08 09:46:...|000468864ee4

In [96]:
# After your write_to_delta function
for table in silver_tables:
    
    print(f"Creating external table: {table}")
    # Creating schema and external table in Spark SQL
    spark.sql("CREATE SCHEMA IF NOT EXISTS silver")
    # Set the current database to 'silver'
    spark.sql("USE silver")
    delta_table_path = f"{silver_location}\\{table.split('.')[1]}"
    spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {table} USING DELTA LOCATION r'{delta_table_path}'")
    #spark.sql(f"SELECT * FROM {table_name}").show()

Creating external table: silver.location
Creating external table: silver.payment
Creating external table: silver.payment_method
Creating external table: silver.payment_status
Creating external table: silver.request
Creating external table: silver.trip
Creating external table: silver.user
Creating external table: silver.vehicle
Creating external table: silver.vehicle_make


In [97]:

# Verify the current schema
print("Current Schema:")
spark.sql("SELECT current_database()").show()

# List all tables in the bronze schema
print("Tables in silver schema:")
spark.sql("SHOW TABLES IN silver").show()
spark.sql("SELECT * FROM silver.location LIMIT 1").show()
spark.sql("SELECT * FROM silver.payment LIMIT 1").show()



Current Schema:
+------------------+
|current_database()|
+------------------+
|            silver|
+------------------+

Tables in silver schema:
+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|   silver|      location|      false|
|   silver|       payment|      false|
|   silver|payment_method|      false|
|   silver|payment_status|      false|
|   silver|       request|      false|
|   silver|          trip|      false|
|   silver|          user|      false|
|   silver|       vehicle|      false|
|   silver|  vehicle_make|      false|
+---------+--------------+-----------+

+-----------+------------+-----------+-------------------+--------------------+--------------------+
|location_id|   longitude|   latitude|    processing_date|record_modified_date|         record_hash|
+-----------+------------+-----------+-------------------+--------------------+--------------------+
|      85038|-73.99800873|40.74245834|2024

In [98]:
from delta.tables import DeltaTable

# Dictionary to store the DeltaTable objects
delta_tables = {}

for table in silver_tables:
    # Extract the file name from the table identifier
    file_name = table.split(".")[1]
    print(file_name)
    
    # Define the table location
    table_location = f"{silver_location}/{file_name}"
    
    # Store the DeltaTable object in the dictionary using the file name as the key
    delta_tables[file_name] = DeltaTable.forPath(spark, table_location)  # Load table from path

    # Check if the table has been vacuumed in the last 30 days
    if delta_tables[file_name].history(30).filter("operation = 'VACUUM START'").count() == 0:
        # Optimize the table for better query performance
        delta_tables[file_name].optimize()
        # Perform vacuum operation (default is to keep data for 7 days)
        delta_tables[file_name].vacuum()


location
payment
payment_method
payment_status
request
trip
user
vehicle
vehicle_make


In [99]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "silver\\location")
fullHistoryDF = deltaTable.history()
fullHistoryDF.show(truncate=False)

+-------+-----------------------+------+--------+------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+-----------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------

# How long do trips take on average, and what is the average distance?

# What are the most common pickup and dropoff locations?

In [ ]:
# Join request_df with location_df for both PickupLocationID and DropoffLocationID
pickup_location_df = request_df.join(location_df, request_df.PickupLocationID == location_df.LocationID)
dropoff_location_df = request_df.join(location_df, request_df.DropoffLocationID == location_df.LocationID)

# Find the most common pickup and dropoff locations
pickup_location_df.groupBy("Longitude", "Latitude").agg(count("*").alias("pickup_count")).orderBy(col("pickup_count").desc()).show()
dropoff_location_df.groupBy("Longitude", "Latitude").agg(count("*").alias("dropoff_count")).orderBy(col("dropoff_count").desc()).show()

# How many trips does each passenger take, and what is their average trip distance?

In [ ]:
# Join trip_df with request_df to get passenger information, then group by passenger
trip_passenger_df = trip_df.join(request_df, trip_df.RequestID == request_df.RequestID)

# Group by PassengerID to calculate the number of trips and average distance
trip_passenger_df.groupBy("PassengerID").agg(
    count("*").alias("trip_count"),
    avg("TripDistance").alias("avg_trip_distance")
).orderBy(col("trip_count").desc()).show()


# What is the average fare and tip per trip by payment method?

# What percentage of trips have different payment statuses?

# What are the most common vehicle makes and models used in trips?

In [ ]:
vehicle_full_df = vehicle_df.join(vehicle_make_df, on='MakeID')

# Group by MakeName and Model, and count the occurrences
vehicle_full_df.groupBy('MakeName', 'Model').count().orderBy('count', ascending=False).show(10)

In [ ]:
trip_df.show(5)